<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
%pip install folium


  Using cached xyzservices-2024.9.0-py3-none-any.whl.metadata (4.1 kB)
Using cached xyzservices-2024.9.0-py3-none-any.whl (85 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium
import pandas as pd

In [4]:

from folium.plugins import MarkerCluster        # Import folium MarkerCluster plugin
from folium.plugins import MousePosition        # Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)

## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
spacex_df['class'].value_counts()

class
0    32
1    24
Name: count, dtype: int64

In [7]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


`.first():`

For each group, retrieves the first row (based on the original DataFrame's order).

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
site_map.add_child(circle)

In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(marker)

In [15]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:green;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


`folium.Circle` expects individual latitude and longitude values, not arrays. To add multiple circles for each launch site, you need to iterate over the DataFrame rows and create a separate folium.Circle for each site.

In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Add a circle for each launch site
for _, site in launch_sites_df.iterrows():
     folium.Circle(
                location = [site['Lat'], site['Long']],
                radius=300, 
                color='#d35400', 
                fill=True,
                fill_color = 'red',
                fill_opacity = 0.7,
                popup = folium.Popup(site['Launch Site'], parse_html = True)
                ).add_to(site_map)
site_map

In [13]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# For each launch site, add a marker object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for _, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=DivIcon(
                    icon_size=(20,20),
                    icon_anchor=(0,0),
                    html='<div style="font-size: 8; color:#d35400;"><b>%s</b></div>' % site['Launch Site']
                    )
               ).add_to(site_map)

# Display the map 
site_map


In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Add a circle for each launch site
for _, site in launch_sites_df.iterrows():
     folium.Circle(
                location = [site['Lat'], site['Long']],
                radius=300, 
                color='#d35400', 
                fill=True,
                fill_color = 'red',
                fill_opacity = 0.7,
                popup = folium.Popup(site['Launch Site'], parse_html = True)
                ).add_to(site_map)

# For each launch site, add a marker object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for _, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=DivIcon(
                    icon_size=(20,20),
                    icon_anchor=(0,0),
                    html='<div style="font-size: 8; color:#d35400;"><b>%s</b></div>' % site['Launch Site']
                    )
               ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>

In [15]:
# Further improvement for better visualization
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Add a marker for each launch site
for _, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=folium.Icon(color='red', icon='info-sign')  # Customize the marker icon
    ).add_to(site_map)

# Display the map 
site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [17]:
marker_cluster = MarkerCluster()
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


`apply Function:`

- Applies a lambda function to each element in the Class column.
- Assigns 'green' for successful launches (Class == 1) and 'red' for unsuccessful launches (Class == 0).

The `lambda` keyword in Python is used to create small, anonymous (unnamed) functions. These functions can take any number of arguments but can only contain a single expression. They are often used in conjunction with functions like map(), filter(), and apply() when you need to define a simple function for a short-term use.

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [18]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [19]:
import folium
from folium.plugins import MarkerCluster

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch site
for _, site in spacex_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=folium.Icon(
                        color='white', 
                        icon_color= site['marker_color'],
                        icon = 'info-sign'
                        )   # Use row-specific marker color
               ).add_to(marker_cluster)


# Display the map 
site_map

### add styled label


In [20]:
import folium
from folium.plugins import MarkerCluster

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch site with both the cluster and styled label
for _, site in spacex_df.iterrows():
    # Add a marker to the cluster
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=folium.Icon(
                        color='white', 
                        icon_color= site['marker_color'],       # Use row-specific marker color
                        icon = 'info-sign'
                        )   
               ).add_to(marker_cluster)

# Add a styled label using DivIcon

    folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
                    icon_size=(150,36),
                    icon_anchor=(0,0),
                    html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
                    )
               ).add_to(site_map)

# Display the map 
site_map

### add circles for each markercluster

In [21]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch site with both the cluster and styled label
for _, site in spacex_df.iterrows():
    # Add a marker to the cluster
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=folium.Icon(
                        color='white', 
                        icon_color= site['marker_color'],       # Use row-specific marker color
                        icon = 'info-sign'
                        )   
               ).add_to(marker_cluster)

    # Add a circle around each marker (with a radius)
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=50,
        color='#d35400',            # Adjusts the border color of the circle.
        fill=True,
        fill_color = '#d35400',
        fill_opacity = 0.02
    ).add_to(site_map)

# Add a styled label using DivIcon

    folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
                    icon_size=(150,36),
                    icon_anchor=(0,0),
                    html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
                    )
               ).add_to(site_map)

# Display the map 
site_map



`Key Parameters:`
- fill_opacity: Adjusts the transparency of the filled color inside the circle. The range is from 0.0 (fully transparent) to 1.0 (fully opaque).
- color: Adjusts the border color of the circle.
- fill_color: Adjusts the color of the circle's filled area.

In [22]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for site_lat, site_long, marker_color in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['marker_color']):
    site_coordinate = [site_lat, site_long]
    marker = folium.map.Marker(
        site_coordinate,
        # Create an icon as a text label
        popup=folium.Popup(site['Launch Site'], parse_html=True),
        icon=folium.Icon(color='white', 
                         icon_color=marker_color,
                         icon = 'info-sign')
    )
    marker.add_to(marker_cluster)
site_map

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


## Notes:
- iterrows() is straightforward but not the most efficient method for large DataFrames because it is slower than vectorized operations.
- For performance-critical tasks, consider using itertuples() or vectorized pandas operations.

In [23]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch site with both the cluster and styled label
for site in spacex_df.itertuples():
    # Add a marker to the cluster
    folium.Marker(
        location=[site.Lat, site.Long],
        popup=folium.Popup(site._1, parse_html=True),  # site._1 corresponds to the Launch Site
        icon=folium.Icon(
            color='white', 
            icon_color=site.marker_color,  # Use row-specific marker color
            icon='info-sign'
        )   
    ).add_to(marker_cluster)

    # Add a circle around each marker (with a radius)
    folium.Circle(
        location=[site.Lat, site.Long],
        radius=50,
        color='#d35400',  # Adjusts the border color of the circle
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.02  # Adjust opacity to see the circle better
    ).add_to(site_map)

    # Add a styled label using DivIcon
    folium.Marker(
        location=[site.Lat, site.Long],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site._1}</b></div>'
        )
    ).add_to(site_map)


# Display the map 
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6378.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [26]:
# find coordinate of the closet coastline
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_site_lat = 28.56341
launch_site_lon = -80.57678
coastline_lat = 28.56289
coastline_lon = -80.56778
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print (distance_coastline, ' Km')

0.8818208312019559  Km


In [27]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
      )
  )
site_map.add_child(distance_marker)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [28]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)


coordinates = [[launch_site_lat, launch_site_lon],[coastline_lat, coastline_lon]] 
lines = folium.PolyLine(locations=coordinates, weight = 1)
site_map.add_child(lines)


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


#### Create a marker with distance to a closest city, railway, highway, etc.
#### Draw a line between the marker to the launch site

In [29]:
# Create a marker with distance to a closest city

# calculate launch_site and Titusville city distance
[launch_site_lat, launch_site_lon] = [28.56341, -80.57678]
[Titusville_lat, Titusville_lon] = [28.60006, -80.81028]
distance_city = calculate_distance(launch_site_lat, launch_site_lon, Titusville_lat, Titusville_lon)
print (distance_city, ' Km')

# add distance marker
distance_marker = folium.Marker(
    [Titusville_lat, Titusville_lon],
    icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
      )
  )
site_map.add_child(distance_marker)

# draw distance line
coordinates = [[launch_site_lat, launch_site_lon],[Titusville_lat, Titusville_lon]]
lines_Titusville = folium.PolyLine(locations=coordinates, weight = 1)
site_map.add_child(lines_Titusville)

23.186716836323974  Km


In [30]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch site with both the cluster and styled label
for site in spacex_df.itertuples():
    # Add a marker to the cluster
    folium.Marker(
        location=[site.Lat, site.Long],
        popup=folium.Popup(site._1, parse_html=True),  # site._1 corresponds to the Launch Site
        icon=folium.Icon(
            color='white', 
            icon_color=site.marker_color,  # Use row-specific marker color
            icon='info-sign'
        )   
    ).add_to(marker_cluster)

    # Add a circle around each marker (with a radius)
    folium.Circle(
        location=[site.Lat, site.Long],
        radius=50,
        color='#d35400',  # Adjusts the border color of the circle
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.02  # Adjust opacity to see the circle better
    ).add_to(site_map)

    # Add a styled label using DivIcon
    folium.Marker(
        location=[site.Lat, site.Long],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site._1}</b></div>'
        )
    ).add_to(site_map)


# Display the map 
site_map

In [33]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

import folium
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distances in kilometers
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6378.0  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Define coordinates
launch_coordinate = [28.562302, -80.57682]

city = [28.61200, -80.80788]
railway = [28.55752, -80.80155]
highway = [28.5402, -80.85079]

# Calculate distances
city_distance = calculate_distance(city[0], city[1], launch_coordinate[0], launch_coordinate[1])
railway_distance = calculate_distance(railway[0], railway[1], launch_coordinate[0], launch_coordinate[1])
highway_distance = calculate_distance(highway[0], highway[1], launch_coordinate[0], launch_coordinate[1])


colors = ['red','orange','green']
html_colors = ['#dc3545','#fd7e14','#198754']

# Add markers and lines for city, railway, highway
for coordinate, distance, color, html_color in zip([city, railway, highway], 
                                                   [city_distance, railway_distance, highway_distance], 
                                                    colors, html_colors):
    # Add marker with distance label
    marker = folium.map.Marker(
            location = coordinate,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 12px; color:{html_color};"><b>{round(distance, 2)} km</b></div>'
                )
            )
    marker.add_to(site_map)

    # Add line connecting to launch site
    folium.PolyLine([coordinate, launch_coordinate], color=color, weight = 1).add_to(site_map)

site_map.add_child(lines)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [34]:
print("City Distance", city_distance)

print("Railway Distance", railway_distance)

print("Highway Distance", highway_distance)

print("Coastline Distance", distance_coastline)

City Distance 23.252981179942925
Railway Distance 21.978695760521973
Highway Distance 26.9014749685055
Coastline Distance 0.8818208312019559


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
